In [ ]:
import evaluate
from py.misс import JSONDataLoader, extract_sbsq
from transformers import AutoTokenizer, T5Tokenizer
import torch
from torch.utils.data import DataLoader

from py.models.model import QAModule
from py.models.data import SquadQADataset

# Параметры модели и данных
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
from py.models.model import QAModule

MODEL_PATH = ".//checkpoints//best_model-epoch=00-val_loss=0.36.ckpt"

tokenizer: T5Tokenizer = T5Tokenizer.from_pretrained(MODEL_PATH, legacy=False)
# Загрузка обученной модели из checkpoint
model = QAModule.load_from_checkpoint(MODEL_PATH)

# Перемещение на GPU/CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Для использования модели:
def use_model(question, context):
    inputs = tokenizer.encode_plus(
        f"[КОНТЕКСТ] {context} [ВОПРОС] {question}",
        truncation=True,
        return_tensors='pt'
    )
    
    with torch.no_grad():
        outputs = model.model.generate(
            inputs['input_ids'].to(device),
            attention_mask=inputs['attention_mask'].to(device)
        )
    
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

In [6]:
# Функция для валидации
def validate_model(model, val_loader, device):
    model.eval()
    metric = evaluate.load("google_bleu")
    

    for batch in val_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        # Извлечение меток
        labels = batch["labels"].to(device)

        # Замена -100 на pad_token_id
        labels = torch.where(labels == -100, tokenizer.pad_token_id, labels)

        with torch.no_grad():
            # Генерация предсказаний
            outputs = model.model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=32)

        # Декодирование предсказаний и меток
        preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        targets = tokenizer.batch_decode(labels, skip_special_tokens=True)

        # Обновление метрики
        for pred, target in zip(preds, targets):
            metric.add(prediction=pred, reference=target)

    return metric.compute()

# Оценка модели на валидационном наборе
results = validate_model(trained_model, val_loader, device)
print("Validation Results:", results)

Using the latest cached version of the module from C:\Users\SawKing\.cache\huggingface\modules\evaluate_modules\metrics\evaluate-metric--google_bleu\6fc70b7be0088120a372dfdd5d320b39b8bb3630cb8029b193941d9376e86bb0 (last modified on Sun Jul 21 18:44:14 2024) since it couldn't be found locally at evaluate-metric--google_bleu, or remotely on the Hugging Face Hub.


KeyError: 0

In [61]:
import evaluate

def generate_answer(
    model: QAModule,
    tokenizer,
    question,
    ref_answer=None,
    device= None,
    text_max_token_len: int = 396
):
    """
    Генерация ответа на вопрос с использованием модели.

    Args:
        model (QAModule): Модель для генерации ответов.
        tokenizer: Токенизатор для преобразования текста.
        question (dict): Словарь с ключами 'question' и 'context'.
        ref_answer (str, optional): Референсный ответ для вычисления метрики. Defaults to None.
        device: Устройство для выполнения (CPU/GPU). Defaults to CPU.
        text_max_token_len (int, optional): Максимальная длина текста. Defaults to 396.

    Returns:
        dict: Содержит предсказанный ответ и, если передан `ref_answer`, BLEU метрику.
    """
    
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)    
    
    # Токенизация вопроса и контекста
    inputs = tokenizer(
        f"[КОНТЕКСТ]{question['context']}[ВОПРОС]|{question['question']}",
        max_length=text_max_token_len,
        padding="max_length",
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors="pt"
    )
    
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    #print(f"Device: {device}")
    #print(f"Model device: {next(model.parameters()).device}")
    #print(f"Inputs device: {inputs['input_ids'].device}")    
    #print(f"attention_mask: {inputs["attention_mask"].device}")
    
    # Генерация ответа
    generated_ids = model.model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        num_beams=2,
        max_length=50,
        repetition_penalty=2.5,
        length_penalty=1.0,
        early_stopping=True,
        use_cache=False,
    )
    
    predicted_answer = tokenizer.decode(
        generated_ids.flatten(),
        skip_special_tokens=True,
        clean_up_tokenization_spaces=True,
    )
    
    # Вычисление BLEU метрики, если есть референсный ответ
    result = {"Предсказанный ответ": predicted_answer}
    if ref_answer:
        try:
            bleu = evaluate.load("google_bleu")
            score = bleu.compute(predictions=[predicted_answer], references=[ref_answer])
            result["Референсный ответ"] = ref_answer
            result["BLEU Score"] = score["google_bleu"]
        except Exception as e:
            result["BLEU Score"] = f"Error computing BLEU: {e}"
    
    # Вывод дополнительной информации
    if ref_answer:
        print("Контекст:\n", question["context"])
        print("\nВопрос:\n", question["question"])
        print("\nПредсказанный ответ:\n", predicted_answer)
        print("\nРеференсный ответ:\n", ref_answer)
        if "BLEU Score" in result:
            print("\nBLEU Score:\n", result["BLEU Score"])
    
    return result


In [66]:
qa_sample = val_df.sample().iloc[0]
generate_answer(trained_model, tokenizer, question = qa_sample, ref_answer = qa_sample["answer_text"])

Контекст:
 Банк был основан предпринимателем Олегом Тиньковым в 2006 году под названием Тинькофф Кредитные Системы . По словам Тинькова, он заинтересовался моделью дистанционного обслуживания американского банка Wells Fargo и монолайнера Capital One, специализирующегося на банковских картах. Вместе с консультантами из Boston Consulting Group он пришёл к выводу, что модель дистанционного кредитного банка может работать в России. Для получения лицензии на банковскую деятельность он приобрёл Химмашбанк , небольшой кэптивный банк, занимавшийся обслуживанием предприятий из химической и фармацевтической отрасли. Предприниматель вложил в открытие банка без отделений 70 млн долларов США из своего восьмидесятимиллионного состояния.

Вопрос:
 Вместе с консультантами из какой компании Тиньков пришёл к выводу, что модель дистанционного кредитного банка может работать в России?

Предсказанный ответ:
 Восточный

Референсный ответ:
 из Boston Consulting Group

BLEU Score:
 0.0


{'Предсказанный ответ': 'Восточный',
 'Референсный ответ': 'из Boston Consulting Group',
 'BLEU Score': 0.0}

In [58]:
my_question = {
    "context": "Русский бандит настиг американского афериста в офисе.",
    "question": "В чём сила?",
    "answer_text": "В правде"
}

generate_answer(trained_model, tokenizer, question= my_question, ref_answer = my_question["answer_text"])

Контекст:
 Русский бандит настиг американского афериста в офисе.

Вопрос:
 В чём сила?

Предсказанный ответ:
 В чём сила

Референсный ответ:
 В правде

BLEU Score:
 0.16666666666666666


{'Предсказанный ответ': 'В чём сила',
 'Референсный ответ': 'В правде',
 'BLEU Score': 0.16666666666666666}